# PV181 Seminar 01 - RNG

This notebook contains python code for several tasks treated in this seminar. 

# Task 0
Look at the old implementation of [rand()](https://code.woboq.org/userspace/glibc/stdlib/random_r.c.html#__random_r)

# Task 1
 Use generic `class PRNG`, create generator (object `gcc_old`) equivalent to old `rand()` and generate (print out) 10 random values.

In [13]:
def Id(x): #lambda x: x is equivalent to Id
    return x

class PRNG:
    def __init__(self, Init = Id, Trans = Id, Out = Id ):
        self.Init = Init
        self.Trans = Trans
        self.Out = Out
        self.state = self.Init(1)

    def srand(self, seed):
        self.state = self.Init(seed)

    def rand(self):
        self.state = self.Trans(self.state)
        return self.Out(self.state)

gcc_old = PRNG()

# Questions 1,2:  
1. What is the problem of the `gcc_old` generator? 
2. Attacker observed that PRNG generated value x. Can he predict **next** or reconstruct **previously** generated values?

# Task 2
Implement "inverse" generator `gcc_old_inv = PRNG()` and generate 10 values.  
**Hint**: `pow(a,-1, m) `computes inverse of $a^{-1} \pmod m$.  

# Task 3
Use monobit test and test the output (at leats 1000 values) of the LCG.

In [14]:
def monobit(bytes):
    num_ones = 0
    for byte in bytes:
        for i in range(8):
            count = (byte >> i) & 1
    n = len(bytes)*8
    sobs = abs(n - 2*num_ones) / math.sqrt(len(bytes))
    p_val = math.erfc(sobs / math.sqrt(2))
    return p_val

pass

# Questions 3:  
How to interpret $p$-value = 0.0001 or 0.4? See  [wiki](https://en.wikipedia.org/wiki/P-value)  or [NIST STS documentation](https://nvlpubs.nist.gov/nistpubs/legacy/sp/nistspecialpublication800-22r1a.pdf)

# Task 4
Use implementation of monobit function and implement `monobit_i(values, i)` which counts only `i`-th bit of each value in the provided list (`values`). Find which of the bits (of 4 bytes) generated by the `gcc_old` is biased. 

In [15]:
def monobit_i(values, i):
    pass

# Questions 4:  
Which of the output bits is biased?  

# Questions 5, 6: 
Following function collects entropy based on time. Why **xor** operation (denoted usually by $\oplus$) of "random" values is not appropriate in this case. What (cryptographic?) function would be more appropriate and why? 

In [ ]:
import time
times = [time.time_ns() for i in range(100)]
res = times[0].to_bytes(8, byteorder='big')
for t in times[1:]:
    res = bytes(a ^ b for (a, b) in zip(res, t.to_bytes(8, byteorder='big')))
print(res.hex())